# Segmenting and Clustering Neighborhoods in Toronto - Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Import csv flie from part2

In [2]:
part2_df = pd.read_csv('df_final.csv',index_col=0)

part2_df.head()

Postcode      Borough                          Neighborhood   Latitude  \
0      M1B  Scarborough                         Rouge,Malvern  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

Check how many boroughs and neighborhoods are in Toronto

In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(part2_df['Borough'].unique()),
        part2_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Get the longitude&latitude of Toronto and Ontario using geolocator

In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create map of Toronto to visualize the neighbourhoods

In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(part2_df['Latitude'], part2_df['Longitude'], part2_df['Borough'], part2_df['Neighborhood']):
    label = '{}, {}'.format(part2_df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color= 'white',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Utilizing the Foursquare API to explore the neighborhoods and segment the

In [23]:
# Define Foursquare Credentials and Version
LIMIT = 100

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: 
CLIENT_SECRET:


Define a function to get all the neighborhoods in Toronto

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Execute the above function on each neighborhood and create a new dataframe called toronto_venues function and get shape of data frame

In [8]:
toronto_venues = getNearbyVenues(names=part2_df['Neighborhood'], latitudes=part2_df['Latitude'], longitudes=part2_df['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

(2283, 7)


Neighborhood  Neighborhood Latitude  \
0                         Rouge,Malvern              43.806686   
1                         Rouge,Malvern              43.806686   
2  Highland Creek,Rouge Hill,Port Union              43.784535   
3       Guildwood,Morningside,West Hill              43.763573   
4       Guildwood,Morningside,West Hill              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.194353            Interprovincial Group       43.805630   
2              -79.160497            Royal Canadian Legion       43.782533   
3              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
4              -79.188711                G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.200378            Print Shop  
2       -79.163085                   Bar  
3       -79.189914           Pizza Place  
4       -79.191537     Electronics Store

Here are the venues for each neighborhood

In [9]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide,King,Richmond                                                100   
Agincourt                                                               5   
Agincourt North,L'Amoreaux East,Milliken,Steele...                      2   
Albion Gardens,Beaumond Heights,Humbergate,Jame...                     11   
Alderwood,Long Branch                                                  11   
Bathurst Manor,Downsview North,Wilson Heights                          19   
Bayview Village                                                         4   
Bedford Park,Lawrence Manor East                                       21   
Berczy Park                                                            58   
Birch Cliff,Cliffside West                                              4   
Bloordale Gardens,Eringate,Markland Wood,Old Bu...                      7   
Brockton,Exhibition Place,Parkdale Village                             24   
Business Reply Mail Processing Centre 969 Eastern                      18   
CFB Toronto,Downsview East                                              2   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                     16   
Cabbagetown,St. James Town                                             49   
Caledonia-Fairbanks                                                     5   
Canada Post Gateway Processing Centre                                  11   
Cedarbrae                                                               8   
Central Bay Street                                                     88   
Chinatown,Grange Park,Kensington Market                               100   
Christie                                                               15   
Church and Wellesley                                                   88   
Clairlea,Golden Mile,Oakridge                                          10   
Clarks Corners,Sullivan,Tam O'Shanter                                  13   
Cliffcrest,Cliffside,Scarborough Village West                           2   
Commerce Court,Victoria Hotel                                         100   
Davisville                                                             37   
Davisville North                                                        8   
Deer Park,Forest Hill SE,Rathnelly,South Hill,S...                     15   
Del Ray,Keelesdale,Mount Dennis,Silverthorn                             6   
Design Exchange,Toronto Dominion Centre                               100   
Don Mills North                                                         5   
Dorset Park,Scarborough Town Centre,Wexford Hei...                      6   
Dovercourt Village,Dufferin                                            17   
Downsview Central                                                       4   
Downsview Northwest                                                     4   
Downsview West                                                          5   
Downsview,North Park,Upwood Park                                        4   
East Birchmount Park,Ionview,Kennedy Park                               6   
East Toronto                                                            4   
Emery,Humberlea                                                         2   
Fairview,Henry Farm,Oriole                                             64   
First Canadian Place,Underground city                                 100   
Flemingdon Park,Don Mills South                                        23   
Forest Hill North,Forest Hill West                                      5   
Glencairn                                                               5   
Guildwood,Morningside,West Hill                                         8   
Harbord,University of Toronto                                          34   
Harbourfront East,Toronto Islands,Union Station                       100   
Harbourfront,Regent Park                        

In [10]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


Analyze Each Neighborhood

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0            0                  0                  0        0   
1            0                  0                  0        0   
2            0                  0                  0        0   
3            0                  0                  0        0   
4            0                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    1   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Basketball Stadium  \
0               0                 0                 0                   0   
1               0                 0                 0                   0   
2               0                 0                 0                   0   
3               0                 0                 0                   0   
4               0                 0                 0                   0   

   Beach  Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  \
0      0                0         0           0                   0   
1      0                0         0           0                   0   
2      0                0         0           0                   0   
3      0                0         0           0                   0   
4      0                0         0           0                   0   

   Bike Shop  Bistro  Board Shop  Boat or Ferry  Bookstore  Boutique  \
0          0       0           0              0          0         0   
1          0       0           0              0          0         0   
2          0       0           0              0          0         0   
3          0       0           0              0          0         0   
4          0       0           0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0    

In [12]:
# Examine the new data frame
toronto_onehot.shape

(2283, 276)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                              Adelaide,King,Richmond     0.000000   
1                                           Agincourt     0.000000   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...     0.000000   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...     0.000000   
4                               Alderwood,Long Branch     0.000000   
5       Bathurst Manor,Downsview North,Wilson Heights     0.000000   
6                                     Bayview Village     0.000000   
7                    Bedford Park,Lawrence Manor East     0.000000   
8                                         Berczy Park     0.000000   
9                          Birch Cliff,Cliffside West     0.000000   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...     0.000000   
11         Brockton,Exhibition Place,Parkdale Village     0.000000   
12  Business Reply Mail Processing Centre 969 Eastern     0.055556   
13                         CFB Toronto,Downsview East     0.000000   
14  CN Tower,Bathurst Quay,Island airport,Harbourf...     0.000000   
15                         Cabbagetown,St. James Town     0.000000   
16                                Caledonia-Fairbanks     0.000000   
17              Canada Post Gateway Processing Centre     0.000000   
18                                          Cedarbrae     0.000000   
19                                 Central Bay Street     0.011364   
20            Chinatown,Grange Park,Kensington Market     0.000000   
21                                           Christie     0.000000   
22                               Church and Wellesley     0.011364   
23                      Clairlea,Golden Mile,Oakridge     0.000000   
24              Clarks Corners,Sullivan,Tam O'Shanter     0.000000   
25      Cliffcrest,Cliffside,Scarborough Village West     0.000000   
26                      Commerce Court,Victoria Hotel     0.000000   
27                                         Davisville     0.000000   
28                                   Davisville North     0.000000   
29  Deer Park,Forest Hill SE,Rathnelly,South Hill,...     0.000000   
30        Del Ray,Keelesdale,Mount Dennis,Silverthorn     0.000000   
31            Design Exchange,Toronto Dominion Centre     0.000000   
32                                    Don Mills North     0.000000   
33  Dorset Park,Scarborough Town Centre,Wexford He...     0.000000   
34                        Dovercourt Village,Dufferin     0.000000   
35                                  Downsview Central     0.000000   
36                                Downsview Northwest     0.000000   
37                                     Downsview West     0.000000   
38                   Downsview,North Park,Upwood Park     0.000000   
39          East Birchmount Park,Ionview,Kennedy Park     0.000000   
40                                       East Toronto     0.000000   
41                                    Emery,Humberlea     0.000000   
42                         Fairview,Henry Farm,Oriole     0.000000   
43              First Canadian Place,Underground city     0.000000   
44                    Flemingdon Park,Don Mills South     0.000000   
45                 Forest Hill North,Forest Hill West     0.000000   
46                                          Glencairn     0.000000   
47                    Guildwood,Morningside,West Hill     0.000000   
48                      Harbord,University of Toronto     0.000000   
49    Harbourfront East,Toronto Islands,Union Station     0.000000   
50                           Harbourfront,Regent Park     0.019231   
51                       High Park,The Junction South     0.000000   
52               Highland Creek,Rouge Hill,Port Union     0.000000   
53                                  Hillcrest Village     0.000000   
54         Humber Bay Shores,Mimico South,New Toronto     0.000000   
55  Humber Bay,King's Mill Park,Kingsway Park Sout...     0.000000   
56                                      Humber Summ

print each neighborhood along with the top 5 most common venues

In [14]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  American Restaurant  0.04
3                  Bar  0.04
4      Thai Restaurant  0.04


----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2                      Yoga Studio   0.0
3                      Men's Store   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0           Pizza Place  0.18
1         Grocery Store  0.18
2              Pharmacy  0.09
3            Beer Store  0.09
4  Fast Food Restaurant  0.09


--

              venue  freq
0    Discount Store  0.33
1  Department Store  0.17
2        Hobby Shop  0.17
3       Bus Station  0.17
4       Coffee Shop  0.17


----East Toronto----
               venue  freq
0               Park  0.50
1  Convenience Store  0.25
2      Metro Station  0.25
3        Yoga Studio  0.00
4        Men's Store  0.00


----Emery,Humberlea----
                             venue  freq
0           Furniture / Home Store   0.5
1                   Baseball Field   0.5
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.14
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3        Cosmetics Shop  0.05
4         Women's Store  0.05


----First Canadian Place,Underground city----
         venue  freq
0  Coffee Shop  0.12
1         Café  0.07
2        Hotel  0.04
3   Restaurant  0.04
4   Steakhouse  0

                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.07
2             Cosmetics Shop  0.04
3       Fast Food Restaurant  0.03
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                             venue  freq
0                       Playground   1.0
1                      Yoga Studio   0.0
2                      Men's Store   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Silver Hills,York Mills----
                             venue  freq
0                Martial Arts Dojo   1.0
1                      Yoga Studio   0.0
2                      Men's Store   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2           Hotel  0.05
3      Restaurant  0.05
4  Clothing Store  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  

Write a function to sort the venues in descending order

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top 10 venues

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighborhood  \
0                              Adelaide,King,Richmond   
1                                           Agincourt   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...   
4                               Alderwood,Long Branch   
5       Bathurst Manor,Downsview North,Wilson Heights   
6                                     Bayview Village   
7                    Bedford Park,Lawrence Manor East   
8                                         Berczy Park   
9                          Birch Cliff,Cliffside West   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...   
11         Brockton,Exhibition Place,Parkdale Village   
12  Business Reply Mail Processing Centre 969 Eastern   
13                         CFB Toronto,Downsview East   
14  CN Tower,Bathurst Quay,Island airport,Harbourf...   
15                         Cabbagetown,St. James Town   
16                                Caledonia-Fairbanks   
17              Canada Post Gateway Processing Centre   
18                                          Cedarbrae   
19                                 Central Bay Street   
20            Chinatown,Grange Park,Kensington Market   
21                                           Christie   
22                               Church and Wellesley   
23                      Clairlea,Golden Mile,Oakridge   
24              Clarks Corners,Sullivan,Tam O'Shanter   
25      Cliffcrest,Cliffside,Scarborough Village West   
26                      Commerce Court,Victoria Hotel   
27                                         Davisville   
28                                   Davisville North   
29  Deer Park,Forest Hill SE,Rathnelly,South Hill,...   
30        Del Ray,Keelesdale,Mount Dennis,Silverthorn   
31            Design Exchange,Toronto Dominion Centre   
32                                    Don Mills North   
33  Dorset Park,Scarborough Town Centre,Wexford He...   
34                        Dovercourt Village,Dufferin   
35                                  Downsview Central   
36                                Downsview Northwest   
37                                     Downsview West   
38                   Downsview,North Park,Upwood Park   
39          East Birchmount Park,Ionview,Kennedy Park   
40                                       East Toronto   
41                                    Emery,Humberlea   
42                         Fairview,Henry Farm,Oriole   
43              First Canadian Place,Underground city   
44                    Flemingdon Park,Don Mills South   
45                 Forest Hill North,Forest Hill West   
46                                          Glencairn   
47                    Guildwood,Morningside,West Hill   
48                      Harbord,University of Toronto   
49    Harbourfront East,Toronto Islands,Union Station   
50                           Harbourfront,Regent Park   
51                       High Park,The Junction South   
52               Highland Creek,Rouge Hill,Port Union   
53                                  Hillcrest Village   
54         Humber Bay Shores,Mimico South,New Toronto   
55  Humber Bay,King's Mill Park,Kingsway Park Sout...   
56                                      Humber Summit   
57                                 Humewood-Cedarvale   
58  Kingsview Village,Martin Grove Gardens,Richvie...   
59  Kingsway Park South West,Mimico NW,The Queensw...   
60                                    L'Amoreaux West   
61                    Lawrence Heights,Lawrence Manor   
62                                      Lawrence Park   
63                                            Leaside   
64                            Little Portugal,Trinity   
65                                   Maryvale,Wexford   
66                         Moore Park,Summerhill East   
67                                 North Toronto West   
68                                          Northwest   
69                     Northwood Park,York Universi

Cluster the Neighborhoods of Downtown Toronto

In [17]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that inclues the cluster as well as the top 10 venues

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_merge = part2_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhood_merge = neighborhood_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhood_merge.head() 

Postcode      Borough                          Neighborhood   Latitude  \
0      M1B  Scarborough                         Rouge,Malvern  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353             0.0  Fast Food Restaurant            Print Shop   
1 -79.160497             4.0                   Bar         Women's Store   
2 -79.188711             0.0     Electronics Store        Medical Center   
3 -79.216917             0.0           Coffee Shop     Korean Restaurant   
4 -79.239476             0.0      Hakka Restaurant                  Bank   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0         Women's Store      Doner Restaurant          Dessert Shop   
1            Donut Shop    Dim Sum Restaurant                 Diner   
2    Mexican Restaurant          Intersection                   Spa   
3         Women's Store             Drugstore    Dim Sum Restaurant   
4    Athletics & Sports                Bakery  Caribbean Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0    Dim Sum Restaurant                 Diner        Discount Store   
1        Discount Store               Dog Run      Doner Restaurant   
2           Pizza Place        Breakfast Spot   Rental Car Location   
3                 Diner        Discount Store               Dog Run   
4       Thai Restaurant   Fried Chicken Joint                Lounge   

  9th Most Common Venue 10th Most Common Venue  
0               Dog Run             Donut Shop  
1             Drugstore       Department Store  
2          Dessert Shop     Dim Sum Restaurant  
3      Doner Restaurant             Donut Shop  
4   Dumpling Restaurant              Drugstore

Create map by dataframe for top 10 venues

In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merge['Latitude'], neighborhood_merge['Longitude'], neighborhood_merge['Neighborhood'], neighborhood_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Cluster 1

In [20]:
neighborhood_merge.loc[neighborhood_merge['Cluster Labels'] == 0, neighborhood_merge.columns[[1] + list(range(5, neighborhood_merge.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
0         Scarborough             0.0       Fast Food Restaurant   
2         Scarborough             0.0          Electronics Store   
3         Scarborough             0.0                Coffee Shop   
4         Scarborough             0.0           Hakka Restaurant   
6         Scarborough             0.0             Discount Store   
7         Scarborough             0.0                   Bus Line   
8         Scarborough             0.0                      Motel   
9         Scarborough             0.0                       Café   
10        Scarborough             0.0          Indian Restaurant   
11        Scarborough             0.0  Middle Eastern Restaurant   
12        Scarborough             0.0                     Lounge   
13        Scarborough             0.0                   Pharmacy   
15        Scarborough             0.0              Grocery Store   
17         North York             0.0                       Pool   
18         North York             0.0             Clothing Store   
19         North York             0.0                       Café   
22         North York             0.0           Ramen Restaurant   
24         North York             0.0               Home Service   
26         North York             0.0       Gym / Fitness Center   
27         North York             0.0                        Gym   
28         North York             0.0                Coffee Shop   
29         North York             0.0         Falafel Restaurant   
31         North York             0.0                       Park   
32         North York             0.0               Home Service   
33         North York             0.0       Gym / Fitness Center   
34         North York             0.0               Hockey Arena   
35          East York             0.0       Fast Food Restaurant   
36          East York             0.0                       Park   
37       East Toronto             0.0                      Trail   
38          East York             0.0                Coffee Shop   
39          East York             0.0          Indian Restaurant   
41       East Toronto             0.0           Greek Restaurant   
42       East Toronto             0.0                Coffee Shop   
43       East Toronto             0.0                       Café   
45    Central Toronto             0.0                       Park   
46    Central Toronto             0.0                Coffee Shop   
47    Central Toronto             0.0                Pizza Place   
49    Central Toronto             0.0                        Pub   
51   Downtown Toronto             0.0                Coffee Shop   
52   Downtown Toronto             0.0                Coffee Shop   
53   Downtown Toronto             0.0                Coffee Shop   
54   Downtown Toronto             0.0                Coffee Shop   
55   Downtown Toronto             0.0                Coffee Shop   
56   Downtown Toronto             0.0                Coffee Shop   
57   Downtown Toronto             0.0                Coffee Shop   
58   Downtown Toronto             0.0                Coffee Shop   
59   Downtown Toronto             0.0                Coffee Shop   
60   Downtown Toronto             0.0                Coffee Shop   
61   Downtown Toronto             0.0                Coffee Shop   
62         North York             0.0                Coffee Shop   
63    Central Toronto             0.0                     Garden   
65    Central Toronto             0.0                       Café   
66   Downtown Toronto             0.0                       Café   
67   Downtown Toronto             0.0                       Café   
68   Downtown Toronto             0.0           Airport Terminal   
69   Downtown Toronto             0.0                Coffee Shop   
70   Downtown Toronto             0.0                Coffee Shop   
71         North York             0.0             Clothing Store   
72

Cluster 2

In [21]:
neighborhood_merge.loc[neighborhood_merge['Cluster Labels'] == 1, neighborhood_merge.columns[[1] + list(range(5, neighborhood_merge.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
23         North York             1.0                  Park   
25         North York             1.0  Fast Food Restaurant   
30         North York             1.0                  Park   
40          East York             1.0                  Park   
44    Central Toronto             1.0                  Park   
50   Downtown Toronto             1.0                  Park   
64    Central Toronto             1.0                  Park   
74               York             1.0                  Park   
79         North York             1.0      Basketball Court   
90          Etobicoke             1.0                  Park   
98               York             1.0     Convenience Store   
100         Etobicoke             1.0                  Park   

    2nd Most Common Venue 3rd Most Common Venue       4th Most Common Venue  \
23                   Bank                   Bar           Convenience Store   
25                   Park     Food & Drink Shop               Women's Store   
30                Airport         Women's Store                   Drugstore   
40      Convenience Store         Metro Station            Doner Restaurant   
44                 Lawyer              Bus Line                 Swim School   
50                  Trail            Playground                    Building   
64                  Trail              Bus Line            Sushi Restaurant   
74          Women's Store  Fast Food Restaurant                      Market   
79                   Park                Bakery  Construction & Landscaping   
90                  River         Women's Store                     Dog Run   
98                   Park            Donut Shop                Dessert Shop   
100           Pizza Place              Bus Line               Women's Store   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23          Women's Store            Donut Shop                 Diner   
25       Doner Restaurant          Dessert Shop    Dim Sum Restaurant   
30     Dim Sum Restaurant                 Diner        Discount Store   
40           Dessert Shop    Dim Sum Restaurant                 Diner   
44       Doner Restaurant    Dim Sum Restaurant                 Diner   
50       Department Store          Dessert Shop    Dim Sum Restaurant   
64          Jewelry Store   Dumpling Restaurant             Drugstore   
74       Doner Restaurant          Dessert Shop    Dim Sum Restaurant   
79          Women's Store                 Diner        Discount Store   
90       Department Store          Dessert Shop    Dim Sum Restaurant   
98     Dim Sum Restaurant                 Diner        Discount Store   
100      Doner Restaurant          Dessert Shop    Dim Sum Restaurant   

    8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
23         Discount Store               Dog Run       Doner Restaurant  
25                  Diner        Discount Store                Dog Run  
30                Dog Run      Doner Restaurant             Donut Shop  
40         Discount Store               Dog Run          Women's Store  
44         Discount Store               Dog Run             Donut Shop  
50                  Diner        Discount Store                Dog Run  
64             Donut Shop      Doner Restaurant          Deli / Bodega  
74                  Diner        Discount Store                Dog Run  
79                Dog Run      Doner Restaurant             Donut Shop  
90                  Diner        Discount Store             Donut Shop  
98                Dog Run      Doner Restaurant          Women's Store  
100                 Diner        Discount Store                Dog Run

Cluster 3

In [ ]:
neighborhood_merge.loc[neighborhood_merge['Cluster Labels'] == 2, neighborhood_merge.columns[[1] + list(range(5, neighborhood_merge.shape[1]))]]

Cluster 4

In [ ]:
neighborhood_merge.loc[neighborhood_merge['Cluster Labels'] == 3, neighborhood_merge.columns[[1] + list(range(5, neighborhood_merge.shape[1]))]]

Cluster 5

In [ ]:
neighborhood_merge.loc[neighborhood_merge['Cluster Labels'] == 4, neighborhood_merge.columns[[1] + list(range(5, neighborhood_merge.shape[1]))]]